In [7]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Descarga desde Citas de Servicios

In [59]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,ParentRecordId from ServiceAppointment
    """
)
Citas_de_servicio = pd.DataFrame(results["records"])


In [38]:
Citas_de_servicio = Citas_de_servicio[
    [
        "Id",
        "Clinica__c",
        "AccountId",
        "CreatedDate",
        "RUT__c",
        "WorkTypeId",
        "Work_TypeGroup__c",
        
        
    ]
]
Citas_de_servicio.shape

(6809, 7)

In [53]:
Citas_de_servicio.head()

,attributes,Id,Clinica__c,AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,ParentRecordId
0,"{'type': 'ServiceAppointment', 'url': '/servic...",08p8c0000007B0zAAE,Clínica de Nutrición,0018c00002azpJ4AAI,2022-12-02T16:23:05.000+0000,183552252,08q8c000000VW1yAAG,0VS8c0000005YZv,Reagendado,Seguimiento,SA-3124,0018c00002azpJ4AAI
1,"{'type': 'ServiceAppointment', 'url': '/servic...",08p8c0000007RPeAAM,Clínica de Nutrición,0018c00002amLbpAAE,2022-12-21T21:28:49.000+0000,224488556,08q8c000000VW1yAAG,0VS8c0000005YZv,Realizado,Seguimiento,SA-4987,0018c00002amLbpAAE
2,"{'type': 'ServiceAppointment', 'url': '/servic...",08p8c0000007P0bAAE,Clínica de Deporte,0018c00002ay6KFAAY,2022-12-12T15:34:28.000+0000,185384799,08q8c000000VW24AAG,0VS8c0000005YaA,Realizado,Explicacion Plan de Intervención,SA-3905,0018c00002ay6KFAAY
3,"{'type': 'ServiceAppointment', 'url': '/servic...",08p8c0000007OumAAE,Clínica de Psicología,0018c00002bI5vJAAS,2022-12-12T12:21:02.000+0000,187821444,08q8c000000VVsBAAW,0VS8c0000005Ya0,Cancelado,Ingreso al programa,SA-3871,0018c00002bI5vJAAS
4,"{'type': 'ServiceAppointment', 'url': '/servic...",08p8c0000007OznAAE,Clínica de Nutrición,0018c00002ao4fCAAQ,2022-12-12T15:04:05.000+0000,167466923,08q8c000000VVs1AAG,0VS8c0000005YZv,Realizado,Ingreso,SA-3898,0018c00002ao4fCAAQ
